In [1]:
def insertionSort(A: list) -> tuple:
    cnt = 0
    for i in range(1, len(A)):
        for j in range(i, 0, -1):
            cnt += 1
            if A[j] < A[j - 1]:
                A[j], A[j - 1] = A[j - 1], A[j]
            else:
                break
    return (A, cnt)

In [2]:
def merge(A: list, B: list) -> tuple:
    a, b, cnt = 0, 0, 0
    ret = []
    while a < len(A) and b < len(B):
        cnt += 1
        if A[a] < B[b]:
            ret.append(A[a])
            a += 1
        else:
            ret.append(B[b])
            b += 1
    while a < len(A):
        ret.append(A[a])
        a += 1
    while b < len(B):
        ret.append(B[b])
        b += 1
    return (ret, cnt)
        
def mergeSort(A: list) -> tuple:
    if len(A) <= 1:
        return (A, 0)
    l, r = 0, len(A) - 1
    mid = (l + r + 1) // 2
    L, cnt_l = mergeSort(A[:mid])
    R, cnt_r = mergeSort(A[mid:])
    A, cnt_merge = merge(L, R)
    return (A, cnt_l + cnt_r + cnt_merge)

In [3]:
def hybridSort(A: list, S: int) -> tuple:
    if len(A) <= S:
        return insertionSort(A)
    elif len(A) <= 1:
        return (A, 0)
    l, r = 0, len(A) - 1
    mid = (l + r + 1) // 2
    L, cnt_l = hybridSort(A[:mid], S)
    R, cnt_r = hybridSort(A[mid:], S)
    A, cnt_merge = merge(L, R)
    return (A, cnt_l + cnt_r + cnt_merge)

In [4]:
def hybridTest(A: list):
    tmp = A.copy()
    print("S\tKey Comparisons")
    for s in range(len(A) + 1):
        A = tmp
        print(f"{s}\t{hybridSort(A, s)[1]}")

In [5]:
A = [5, 2, 4, 2, 1]
insertionSort(A)

([1, 2, 2, 4, 5], 10)

In [6]:
A = [14, 40, 31, 28, 3, 15, 17, 51]
mergeSort(A)

([3, 14, 15, 17, 28, 31, 40, 51], 16)

In [7]:
hybridTest(A)

S	Key Comparisons
0	16
1	16
2	16
3	16
4	16
5	16
6	16
7	16
8	19


In [8]:
A = [23, 23, 23, 23, 23, 23, 23, 23]
mergeSort(A)

([23, 23, 23, 23, 23, 23, 23, 23], 12)

In [9]:
hybridTest(A)

S	Key Comparisons
0	12
1	12
2	12
3	12
4	10
5	10
6	10
7	10
8	7
